- first part of codebase: model dict, pred loop etc
- get model output
- on public data (=codebase) and on private data
- TODO check model outputs, not correlated as of now

In [ ]:
%%capture output
!pip install --upgrade pip
!pip install --upgrade pandas
!pip install tables   
# necessary for pd.read_hdf()

!pip install ipywidgets
!pip install --upgrade jupyter
!pip install IProgress
!pip install catboost
!pip install shap
!pip install anndata

In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
%matplotlib inline
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)

## data load

In [ ]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/open-problems-multimodal-3rd-solution/'

# model_path_for_now = '/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/'

raw_path =  lrz_path + 'input/raw/'  # '../../../input/raw/'

cite_target_path = lrz_path + 'input/target/cite/'   # '../../../input/target/cite/'
cite_feature_path = lrz_path + 'input/features/cite/'   # '../../../input/features/cite/'
cite_mlp_path = lrz_path + 'model/cite/mlp/'   # '../../../model/cite/mlp/'   # '../../../model/cite/mlp/'
cite_cb_path = lrz_path + 'model/cite/cb/'   # '../../../model/cite/cb/'

# multi_target_path = lrz_path + 'input/target/multi/'   # '../../../input/target/multi/'
# multi_feature_path = lrz_path + 'input/features/multi/'   # '../../../input/features/multi/'
# multi_mlp_path = lrz_path + 'model/multi/mlp/'   # '../../../model/multi/mlp/'
# multi_cb_path = lrz_path + 'model/multi/cb/'   # '../../../model/multi/cb/'

index_path = lrz_path + 'input/preprocess/cite/'

output_path = lrz_path + 'output/'   # '../../../output/'

## Cite

In [ ]:
# get model name
#mlp_model_path = os.listdir(cite_mlp_path)

markdown

In [ ]:
# check model names and lists/dict/...

In [ ]:
mlp_model_name = [
    'corr_add_con_imp',
    'corr_last_v3', 
    'corr_c_add_w2v_v1_mish_flg',
    'corr_c_add_w2v_v1_flg',
    'corr_c_add_84_v1',
    'corr_c_add_120_v1',
    'corr_w2v_cell_flg',
    'corr_best_cell_120',
    'corr_cluster_cell',
    'corr_w2v_128',
    'corr_imp_w2v_128',
    'corr_snorm',
    'corr_best_128',
    'corr_best_64',
    'corr_cluster_128',
    'corr_cluster_64',
    'corr_svd_128',
    'corr_svd_64',
             ]

In [ ]:
model_name_list = []

for i in mlp_model_name:
    for num, j in enumerate(os.listdir(cite_mlp_path)):
        if i in j:
            model_name_list.append(j)

len(model_name_list)
model_name_list   # TODO why not listdir(cite_mlp_path) directly? always 18 models... does mlp_model_name list come from somewhere else?

In [ ]:
weight = [1, 0.3, 1, 1, 1, 1, 1, 1, 1, 0.8, 0.8, 0.8, 0.8, 0.5, 0.5, 0.5, 1, 1, 2, 2]
weight_sum = np.array(weight).sum()    # == 19

# model_feat_dict = {model_name_list[0]:['X_test_add_con_imp.pickle', 1],
#                    model_name_list[1]:['X_test_last_v3.pickle', 0.3],
#                    model_name_list[2]:['X_test_c_add_w2v_v1.pickle', 1],
#                    model_name_list[3]:['X_test_c_add_w2v_v1.pickle', 1],
#                    model_name_list[4]:['X_test_c_add_84_v1.pickle', 1],
#                    model_name_list[5]:['X_test_c_add_v1.pickle', 1],
                   
#                    model_name_list[6]:['X_test_feature_w2v_cell.pickle', 1],
#                    model_name_list[7]:['X_test_best_cell_128_120.pickle', 1],
#                    model_name_list[8]:['X_test_cluster_cell_128.pickle', 1],
                   
#                    model_name_list[9]:['X_test_feature_w2v.pickle', 0.8],
#                    model_name_list[10]:['X_test_feature_imp_w2v.pickle',0.8],
#                    model_name_list[11]:['X_test_feature_snorm.pickle', 0.8],
#                    model_name_list[12]:['X_test_best_128.pickle', 0.8],
#                    model_name_list[13]:['X_test_best_64.pickle', 0.5],
#                    model_name_list[14]:['X_test_cluster_128.pickle', 0.5],
#                    model_name_list[15]:['X_test_cluster_64.pickle', 0.5],
#                    model_name_list[16]:['X_test_svd_128.pickle', 1],
#                    model_name_list[17]:['X_test_svd_64.pickle', 1],
                   
#                    'best_128':['X_test_best_128.pickle', 2],
#                    'best_64':['X_test_best_64.pickle', 2],
#                   }

In [ ]:
# create list of file names to have model_feat_dict in a cleaner way... but is it still readable? TODO
file_names = ['X_test_add_con_imp.pickle',
 'X_test_last_v3.pickle',
 'X_test_c_add_w2v_v1.pickle',
 'X_test_c_add_w2v_v1.pickle',
 'X_test_c_add_84_v1.pickle',
 'X_test_c_add_v1.pickle',
 'X_test_feature_w2v_cell.pickle',
 'X_test_best_cell_128_120.pickle',
 'X_test_cluster_cell_128.pickle',
 'X_test_feature_w2v.pickle',
 'X_test_feature_imp_w2v.pickle',
 'X_test_feature_snorm.pickle',
 'X_test_best_128.pickle',
 'X_test_best_64.pickle',
 'X_test_cluster_128.pickle',
 'X_test_cluster_64.pickle',
 'X_test_svd_128.pickle',
 'X_test_svd_64.pickle',
 'X_test_best_128.pickle',
 'X_test_best_64.pickle']

model_feat_dict = {name: [file_name, weight] for name, weight, file_name in zip(model_name_list+['best_128', 'best_64'], weight, file_names)}


### cite model

In [ ]:
def std(x):
    x = np.array(x)
    return (x - x.mean(1).reshape(-1, 1)) / x.std(1).reshape(-1, 1)

In [ ]:
# class CiteDataset(Dataset):
    
#     def __init__(self, feature, target):
        
#         self.feature = feature
#         self.target = target
        
#     def __len__(self):
#         return len(self.feature)
    
#     def __getitem__(self, index):
                
#         d = {
#             "X": self.feature[index],
#             "y" : self.target[index],
#         }
#         return d

In [ ]:
class CiteDataset_test(Dataset):
    
    def __init__(self, feature):
        self.feature = feature
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index]
        }
        return d

In [ ]:
# def partial_correlation_score_torch_faster(y_true, y_pred):
#     """Compute the correlation between each rows of the y_true and y_pred tensors.
#     Compatible with backpropagation.
#     """
#     y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
#     y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
#     cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
#     var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
#     var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
#     return cov_tp/torch.sqrt(var_t*var_p)

# def correl_loss(pred, tgt):
#     """Loss for directly optimizing the correlation.
#     """
#     return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))

In [ ]:
class CiteModel(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.ReLU(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.ReLU(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.ReLU(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
        
        from_numpy = False
        
      ##
        if isinstance(X, np.ndarray):
            X = torch.from_numpy(X)
            from_numpy = True
        X = X.to(device)  # Move the input to the appropriate device if necessary
        ##
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        if from_numpy:
            out = out.cpu().detach().numpy()
            
        return out

In [ ]:
class CiteModel_mish(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel_mish, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.Mish(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.Mish(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.Mish(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
    
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [ ]:
# def train_loop(model, optimizer, loader, epoch):
    
#     losses, lrs = [], []
#     model.train()
#     optimizer.zero_grad()
#     #loss_fn = nn.MSELoss()
    
#     with tqdm(total=len(loader),unit="batch") as pbar:
#         pbar.set_description(f"Epoch{epoch}")
        
#         for d in loader:
#             X = d['X'].to(device)
#             y = d['y'].to(device)
            
#             logits = model(X)
#             loss = correl_loss(logits, y)
#             #loss = torch.sqrt(loss_fn(logits, y))
        
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             pbar.set_postfix({"loss":loss.item()})
#             pbar.update(1)

#     return model

In [ ]:
# def valid_loop(model, loader, y_val):
    
#     model.eval()
#     partial_correlation_scores = []
#     oof_pred = []
    
#     for d in loader:
#         with torch.no_grad():
#             val_X = d['X'].to(device).float()
#             val_y = d['y'].to(device)
#             logits = model(val_X)
#             oof_pred.append(logits)
    
#     #print(torch.cat(oof_pred).shape, torch.cat(oof_pred).detach().cpu().numpy().shape)
#     cor = partial_correlation_score_torch_faster(torch.tensor(y_val).to(device), torch.cat(oof_pred))
#     cor = cor.mean().item()
#     logits = torch.cat(oof_pred).detach().cpu().numpy()
    
#     return logits, cor

In [ ]:
def test_loop(model, loader):
    
    model.eval()
    predicts=[]

    for d in tqdm(loader):
        with torch.no_grad():
            X = d['X'].to(device)
            logits = model(X)
            predicts.append(logits.detach().cpu().numpy())
            
    return np.concatenate(predicts)

### ensemble prediction: use all models, sum weighted predictions

In [ ]:
pred = np.zeros([48203, 140])

for num, i in enumerate(model_feat_dict.keys()):
    
    print(i)
    
    if 'mlp' in i:

        try:
            test_file = model_feat_dict[i][0]
            test_weight = model_feat_dict[i][1]
            X_test = pd.read_pickle(cite_feature_path  + test_file)   
            # print(cite_feature_path  + test_file)
            X_test = np.array(X_test)
            feature_dims = X_test.shape[1]

            test_ds = CiteDataset_test(X_test)
            test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                         shuffle=False, drop_last=False, num_workers=4)

            if 'mish' in i:
                model = CiteModel_mish(feature_dims)
            else:
                model = CiteModel(feature_dims)

            model = model.to(device)
            model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

            result = test_loop(model, test_dataloader).astype(np.float32)
            result = std(result) * test_weight / weight_sum
            pred += result

            torch.cuda.empty_cache()

        except Exception as e: 
            print(i)
            print(e)             # TODOOOOOOOOOOOOOO
        
    else:
        test_file = model_feat_dict[i][0]
        test_weight = model_feat_dict[i][1]
        X_test = pd.read_pickle(cite_feature_path  + test_file)
        
        cb_pred = np.zeros([48203, 140])
        
        for t in tqdm(range(140)): 
            cb_model_path = [j for j in os.listdir(cite_cb_path) if f'cb_{t}_{i}' in j][0]
            cb = pickle.load(open(cite_cb_path + cb_model_path, 'rb'))
            cb_pred[:,t] = cb.predict(X_test)
            
        cb_pred = cb_pred.astype(np.float32)
        pred += std(cb_pred) * test_weight / weight_sum
        
        del cb_pred

In [ ]:
pd.DataFrame(pred)

In [ ]:
# TODO check pred / target

### pred loop function (only for single models, not ensemble)

In [ ]:
def get_pred(model_name, X_test=None): 

    '''if test file from model_feat_dict: pass X_test=None
    if other test file, e.g. private test data, pass it as param '''


    # pred = np.zeros([rows, cols])
    if X_test is None:
        test_file = model_feat_dict[model_name][0]
        X_test = np.array(pd.read_pickle(cite_feature_path + test_file))
    
    feature_dims = X_test.shape[1]
    print(feature_dims)

    test_ds = CiteDataset_test(np.array(X_test))
    test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                                shuffle=False, drop_last=False, num_workers=4)

    if 'mish' in model_name:
        model = CiteModel_mish(feature_dims)
    else:
        model = CiteModel(feature_dims)
        
    model = model.to(device)
    model.load_state_dict(torch.load(f'{cite_mlp_path}/{model_name}'))

    pred = test_loop(model, test_dataloader).astype(np.float32)
   
    torch.cuda.empty_cache()
            
    return pd.DataFrame(pred)

use for model 16:

In [ ]:
# model #16: cite_mlp_corr_svd_128_flg_donor_val_30
get_pred('cite_mlp_corr_svd_128_flg_donor_val_30')

# double check train_cite_targets.h5  -> omnipath

In [ ]:
pd.read_hdf('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/train_cite_targets.h5')

use for model 17:

In [ ]:
# model #17: 'cite_mlp_corr_svd_64_flg_donor_val_38'
get_pred('cite_mlp_corr_svd_64_flg_donor_val_38')

### prediction with private test input -> should get private test target

In [ ]:
private_test_input = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_input.h5ad')
private_test_input

In [ ]:
private_test_target = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_target.h5ad')
private_test_target

In [ ]:
# private_test_input.to_df()

In [ ]:
# should be non-raw df (non-int)
# private_test_target_raw = ad.read('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/large_preprocessed_files/private_test_target_raw.h5ad')
# private_test_target_raw

In [ ]:
with open('private_X_test_svd.pkl', 'rb') as f:  # private_X_test_svd

    private_X_test_svd = pickle.load(f)
private_X_test_svd.shape

In [ ]:
# with open('private_X_test_svd_from_raw.pkl', 'rb') as f:  # private_X_test_svd

#     private_X_test_svd_from_raw = pickle.load(f)
# private_X_test_svd_from_raw.shape

In [ ]:
# get model #16 prediction on private test data
# model #16: cite_mlp_corr_svd_128_flg_donor_val_30

pred_private = get_pred('cite_mlp_corr_svd_128_flg_donor_val_30', private_X_test_svd)
print(pred_private.shape)
pred_private.head()

In [ ]:
pd.DataFrame(private_test_target.X).head()

In [ ]:
# pd.DataFrame(private_test_target_raw.X.toarray())

In [ ]:
pd.concat([pd.DataFrame(pred_private), pd.DataFrame(private_test_target.X)]).corr().head()   # TODO is this correct? this compares all columns but not both datasets?